In [2]:
import tensorflow as tf

tf.__version__

'2.19.0'

In [3]:
import numpy as np
import os
import time

In [5]:
# dowload the file
data = tf.keras.utils.get_file("shakespeare.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# read the data
with open(data, "r") as f:
    text = f.read()
    

In [9]:
len(text)

1115394

In [11]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [18]:
# ubique char in the text
vocab = sorted(list(set(text)))

len(vocab)

65

In [51]:
char_to_inx = tf.keras.layers.StringLookup(vocabulary=vocab)

In [54]:
char_to_inx("A")

<tf.Tensor: shape=(), dtype=int64, numpy=14>

In [56]:
inx_to_char = tf.keras.layers.StringLookup(vocabulary=vocab, invert=True)


# vocabulary = string , vocab is list(str)

In [57]:
inx_to_char(14)

<tf.Tensor: shape=(), dtype=string, numpy=b'A'>

In [58]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [60]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [79]:
def text_from_ids(ids):
  return tf.strings.reduce_join(inx_to_char(ids), axis=-1)

In [85]:
all_ids = char_to_inx(tf.strings.unicode_split(text, 'UTF-8'))

In [86]:
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [87]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [90]:
for i in ids_dataset.take(5):
    print(i.numpy())

19
48
57
58
59


In [91]:
seq_length = 100


In [92]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

sequences

<_BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [98]:
for seq in sequences.take(1):
  print(inx_to_char(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [96]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [107]:
ss = "tensorflow"

ss[0:-1]

'tensorflo'

In [108]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [109]:
dataset = sequences.map(split_input_target)
dataset

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [112]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [113]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [118]:
# Build the moddel


vocab_size = len(char_to_inx.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
from tensorflow.keras import Model
class MyModel(Model):
    def __init__(self, vocab_size, rnn_units, embedding_dim):
        super().__init__(self)
        

In [148]:
import torch
from torch import nn


a = nn.Linear(in_features=2, out_features=2, bias=False)

b =torch.tensor([[1.0,  2.0],
                 [1.0,  2.0],
                 [1.0,  2.0]])

a(b)


tensor([[-0.7068, -0.2099],
        [-0.7068, -0.2099],
        [-0.7068, -0.2099]], grad_fn=<MmBackward0>)

In [149]:
b.shape

torch.Size([3, 2])

In [150]:
a.state_dict()

OrderedDict([('weight',
              tensor([[ 0.2864, -0.4966],
                      [ 0.0372, -0.1235]]))])

In [156]:
b.size(0)

3

In [161]:
torch.tril(torch.ones(1,2))

tensor([[1., 0.]])